<h1>Approach</h1>
Use CountVectorizer with cleaned and tokenized text with Naive Bayes

In [31]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
from sklearn import naive_bayes, metrics
from keras.layers import Dropout
from keras.models import Sequential
from keras.layers import Dense

In [4]:
isear=pd.read_csv('isear_1.csv', header=-1)
isear=isear.drop(2, axis=1)
isear[0][isear[0]=='guit']='guilt'

7516

In [24]:
negative = ['not', 'neither', 'nor', 'but', 'however', 'although', 'nonetheless', 'despite', 'except',
                         'even though', 'yet']
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)

In [22]:
def clean(doc):
    lemma=WordNetLemmatizer()
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop if i not in negative])
    punc_free = "".join([ch for ch in stop_free if ch not in exclude])
    normalized = " ".join([lemma.lemmatize(word) for word in punc_free.split()])
    return normalized

<h3>Only use below for certain implementations of Naive Bayes</h3>

In [ ]:
text=[]
for index, row in isear.iterrows():
    text.append(clean(row[1]))

In [41]:
X=isear[1].apply(clean)
y=isear[0]

In [42]:
encoder=LabelEncoder()
y=encoder.fit_transform(y)
y=np_utils.to_categorical(y)

In [43]:
train_x, valid_x, train_y, valid_y = train_test_split(X, y, test_size = 0.2, random_state = 0)

<h3>Below step only for non-neural network</h3>

In [ ]:
train_y=encoder.fit_transform(train_y)
valid_y=encoder.fit_transform(valid_y)

In [45]:
# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(X)

# transform the training and validation data using count vectorizer object
xtrain_count =  count_vect.transform(train_x)
xvalid_count =  count_vect.transform(valid_x)

In [46]:
xtrain_count

<6012x8511 sparse matrix of type '<class 'numpy.int64'>'
	with 58704 stored elements in Compressed Sparse Row format>

In [17]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return metrics.accuracy_score(predictions, valid_y)

In [30]:
# Naive Bayes on Count Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_count, train_y, xvalid_count)
print("NB, Count Vectors: ", accuracy)

NB, Count Vectors:  0.5485372340425532


<h2>Outcome of Naive Bayes</h2>

NB, Count Vectors:  Accuracy of 0.5485372340425532
Now try neural network

In [48]:
#without wrapper
classifier=Sequential()
# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 8511))

# Adding the second hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))

#Adding dropout
classifier.add(Dropout(0.4, seed=0))

#Adding the third hidden layer
classifier.add(Dense(units=10, kernel_initializer='uniform', activation='relu'))

#Adding dropout
classifier.add(Dropout(0.4, seed=0))

# Adding the output layer (Softmax because we want probabilities within 0 and 1 )
classifier.add(Dense(units = 7, kernel_initializer = 'uniform', activation = 'softmax'))

# Compiling the ANN (gradient descent with logarithmic loss)
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [51]:
classifier.fit(xtrain_count, train_y, batch_size = 10, epochs = 100)

Epoch 1/100
6012/6012 [==============================] - 2s 312us/step - loss: 1.2936 - acc: 0.4892
Epoch 2/100
6012/6012 [==============================] - 2s 312us/step - loss: 1.3195 - acc: 0.4854
Epoch 3/100
6012/6012 [==============================] - 2s 312us/step - loss: 1.3029 - acc: 0.4877
Epoch 4/100
6012/6012 [==============================] - 2s 313us/step - loss: 1.3010 - acc: 0.4840
Epoch 5/100
6012/6012 [==============================] - 2s 316us/step - loss: 1.3002 - acc: 0.4920
Epoch 6/100
6012/6012 [==============================] - 2s 311us/step - loss: 1.2870 - acc: 0.4917
Epoch 7/100
6012/6012 [==============================] - 2s 315us/step - loss: 1.3053 - acc: 0.4879
Epoch 8/100
6012/6012 [==============================] - 2s 316us/step - loss: 1.2840 - acc: 0.4977
Epoch 9/100
6012/6012 [==============================] - 2s 330us/step - loss: 1.2935 - acc: 0.4797
Epoch 10/100
6012/6012 [==============================] - 2s 317us/step - loss: 1.2803 - acc: 0.4933

6012/6012 [==============================] - 2s 308us/step - loss: 1.3779 - acc: 0.4779
Epoch 83/100
6012/6012 [==============================] - 2s 306us/step - loss: 1.3802 - acc: 0.4854
Epoch 84/100
6012/6012 [==============================] - 2s 303us/step - loss: 1.3873 - acc: 0.4800
Epoch 85/100
6012/6012 [==============================] - 2s 303us/step - loss: 1.3886 - acc: 0.4890
Epoch 86/100
6012/6012 [==============================] - 2s 305us/step - loss: 1.3783 - acc: 0.4837
Epoch 87/100
6012/6012 [==============================] - 2s 308us/step - loss: 1.3740 - acc: 0.4760
Epoch 88/100
6012/6012 [==============================] - 2s 303us/step - loss: 1.3420 - acc: 0.4877
Epoch 89/100
6012/6012 [==============================] - 2s 305us/step - loss: 1.3706 - acc: 0.4772
Epoch 90/100
6012/6012 [==============================] - 2s 300us/step - loss: 1.3674 - acc: 0.4847
Epoch 91/100
6012/6012 [==============================] - 2s 306us/step - loss: 1.3624 - acc: 0.4844
Epo

<h3>Results of ANN</h3>
~.49 accuracy
Now try LSTM